In [ ]:
import os, sys
import glob
import json
import string

import datetime as dt

import numpy as np
import xarray as xr
import pandas as pd
import networkx as nx

import geopandas as gpd
from shapely.affinity import affine_transform

import seaborn as sns
import matplotlib as mpl

from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

project_root = os.path.abspath(os.pardir)
python_path = os.path.join(project_root, "python")
sys.path.append(python_path)

from plotutils import *
from metricutils import *

In [ ]:
# Custom parameters for the plots
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="whitegrid", palette="deep", rc=custom_params)

In [ ]:
############################
# Loading all the data need
############################

layer               = "abs_09"
run_id              = "run_w14_d7_R_risk_ij"
experiments_folder  = "../outputs/"

output_folder       = os.path.join(experiments_folder, f"TE_{layer}/")
config_fname        = os.path.join(output_folder, "config.json")

te_run_folder       = os.path.join(output_folder, run_id)
te_output_fname     = os.path.join(te_run_folder, "TE.nc")
te_params_fname     = os.path.join(te_run_folder, f"params.json")

print(config_fname)
with open(config_fname) as fh:
    config = json.load(fh)


population_ds_fname = config["cases_ds_fname"]
mobility_ds_fname  = config["risk_ds_fname"]

with open(te_params_fname) as fh:
    te_params = json.load(fh)
    te_params = te_params['te_params']

Txy           = xr.load_dataarray(te_output_fname)
TSxy          = calculate_net_TSxy(Txy)
population_ds = xr.load_dataset(os.path.join(experiments_folder, f"TE_{layer}/cases_pop_ds.nc"))
mobility_ds  = xr.load_dataset(os.path.join(experiments_folder, f"TE_{layer}/risk_ds.nc"))
te_x_varname  = te_params["TE_x"]
te_y_varname  = te_params["TE_y"]
date_range    = Txy.coords['date'].values


# Creating geopandas dataframe to plot the maps
data_folder = os.path.join(project_root, config["data_folder"])
geojson_fname  = os.path.join(data_folder, f"{layer}.geojson")
gdf_patches = gpd.read_file(geojson_fname)
gdf_patches.set_index('id', inplace=True)
gdf_patches['centroid'] = gdf_patches['geometry'].apply(lambda x: x.centroid)
gdf_patches = gdf_patches[~gdf_patches.index.isin(config['filter_patches'])]
gdf_patches = gdf_patches.sort_index()

In [ ]:
# 1 - Regió sanitària RS11 Alt Pirineu i Aran
# 2 - Regió sanitària RS12 Barcelona
# 3 - Regió sanitària RS13 Camp de Tarragona
# 4 - Regió sanitària RS14 Catalunya Central
# 5 - Regió sanitària RS15 Girona
# 6 - Regió sanitària RS16 Lleida
# 7 - Regió sanitària RS17 Terres de l'Ebre

reg_code = 2

sectores = pd.read_csv("../data/sanitat.csv")
sel_dict = {}
c = 0
sel = []
for index, row in sectores.iterrows():
    if row.Nivell == "Regió sanitària":
        if c > 0:
            sel_dict[c] = sel
        c += 1
        sel = []
    elif row.Nivell == "Àrea bàsica salut":
        cod = row.Codi.zfill(3)
        #if "Barcelona" in row.Nom:
        if cod in gdf_patches.index and cod not in ["381", "379", "377"]:
            sel.append(cod)
    else:
        continue

sel_dict[c] = sel
sel_ids = sel_dict[reg_code]

In [ ]:
layer               = "zon_bas_13"
run_id              = "run_w14_d7_R_risk_ij"
experiments_folder  = "../outputs/"

output_folder       = os.path.join(experiments_folder, f"TE_{layer}/")
config_fname        = os.path.join(output_folder, "config.json")

te_run_folder       = os.path.join(output_folder, run_id)
te_output_fname     = os.path.join(te_run_folder, "TE.nc")
te_params_fname     = os.path.join(te_run_folder, f"params.json")

print(config_fname)
with open(config_fname) as fh:
    config = json.load(fh)


population_ds_fname = config["cases_ds_fname"]
mobility_ds_fname  = config["risk_ds_fname"]

with open(te_params_fname) as fh:
    te_params = json.load(fh)
    te_params = te_params['te_params']

Txy2           = xr.load_dataarray(te_output_fname)
TSxy2 = calculate_net_TSxy(Txy2)
population_ds2 = xr.load_dataset(os.path.join(experiments_folder, f"TE_{layer}/cases_pop_ds.nc"))
mobility_ds2   = xr.load_dataset(os.path.join(experiments_folder, f"TE_{layer}/risk_ds.nc"))

In [ ]:
import matplotlib.dates as mdates
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", palette="deep", rc=custom_params)

fig, ax = plt.subplots(4, 1, figsize=(10, 10), dpi=300, sharex=True)
fig.set_facecolor('white')

date_range = Txy.coords['date'].values

date_start = date_range[200]
date_end = date_range[283]

trips = mobility_ds["trips_ij"]
risk = mobility_ds["risk_ij"]
cases = population_ds.new_cases_by_100k

pop1 = population_ds.total_population.sum("id").loc[date_start:date_end]
pop2 = population_ds2.total_population.sum("id").loc[date_start:date_end]
    
data = TSxy.loc[:,:,date_start:date_end].mean(["source", "target"]).to_pandas()
ax[3].plot(data,label="Catalunya", color="r", linewidth=2)

data = TSxy2.loc[:,:,date_start:date_end].mean(["source", "target"]).to_pandas()
ax[3].plot(data, label="Madrid", color="b", linestyle="--", linewidth=2)

data = (trips.loc[:,:,date_start:date_end].sum(["source", "target"])/pop1).to_pandas()
ax[0].plot(data,label="Catalunya", color="r", linewidth=2)

data = (mobility_ds2["trips_ij"].loc[:,:,date_start:date_end].sum(["source", "target"])/pop2).to_pandas()
ax[0].plot(data, label="Madrid", color="b", linestyle="--", linewidth=2)

data = risk.loc[:,:,date_start:date_end].mean(["source", "target"]).to_pandas()
ax[2].plot(data,label="Catalunya", color="r", linewidth=2)

data = mobility_ds2["risk_ij"].loc[:,:,date_start:date_end].mean(["source", "target"]).to_pandas()
ax[2].plot(data, label="Madrid", color="b", linestyle="--", linewidth=2)

data = cases.loc[:,date_start:date_end].mean("id").to_pandas()
ax[1].plot(data, label="Catalunya", color="r", linewidth=2)

data = population_ds2.new_cases_by_100k.loc[:,date_start:date_end].mean("id").to_pandas()
ax[1].plot(data, label="Madrid", color="b", linestyle="--", linewidth=2)


a = date_range[229]
b = date_range[269]
c = date_range[215]
d = date_range[255]

ax[3].axvspan(a, b, facecolor='grey', alpha=0.2)
ax[3].axvline(c,linestyle="dotted", color="k", alpha=0.4, linewidth=2)
ax[3].axvline(d,linestyle="dotted", color="k", alpha=0.4, linewidth=2)


for i in range(3):
    ax[i].axvspan(c, d, facecolor='grey', alpha=0.2)
    
ax[3].set_ylabel(r"$DI^+(t)$")
ax[0].set_ylabel(r"$M(t)$ per person")
ax[2].set_ylabel(r"$R(t)$")
ax[1].set_ylabel(r"$I(t)$")
ax[3].xaxis.set_major_locator(mdates.DayLocator(interval=14))
ax[3].xaxis.set_major_formatter(mdates.DateFormatter("%d/%m"))

for i in range(4):
    ax[i].set_xlabel("")
ax[0].legend()

fig.savefig("../figures/main/Figure6.tiff", dpi=300)

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

dates = [date_range[229], date_range[269]]
di0_bar = np.array(TSxy.loc[:,:,dates[0]].where(TSxy.loc[:,:,dates[0]] > 0)/pop1).flatten()
di1_bar = np.array(TSxy.loc[:,:,dates[1]].where(TSxy.loc[:,:,dates[1]] > 0)/pop1).flatten()

fig, ax = plt.subplots()

data0_bar = di0_bar[np.logical_not(np.isnan(di0_bar))]
ax.hist(data0_bar, label="before policy")

data1_bar = di1_bar[np.logical_not(np.isnan(di1_bar))]
ax.hist(data1_bar, label="after policy")

ax.set_ylabel("counts")
ax.set_xlabel(r"$DI^+_{xy}$")
ax.legend()
fig.set_facecolor('white')

print(mannwhitneyu(data0_bar, data1_bar))

In [ ]:
fig, ax = plt.subplots()

di0_mad = np.array(TSxy2.loc[:,:,dates[0]].where(TSxy2.loc[:,:,dates[0]] > 0)/pop2).flatten()
di1_mad = np.array(TSxy2.loc[:,:,dates[1]].where(TSxy2.loc[:,:,dates[1]] > 0)/pop2).flatten()

data0_mad = di0_mad[np.logical_not(np.isnan(di0_mad))]
ax.hist(data0_mad, label="before policy")

data1_mad = di1_mad[np.logical_not(np.isnan(di1_mad))]
ax.hist(data1_mad, label="after policy")

ax.set_ylabel("counts")
ax.set_xlabel(r"$DI^+_{xy}$")
ax.legend()
fig.set_facecolor('white')

print(mannwhitneyu(data0_mad, data1_mad))

In [ ]:
from statannotations.Annotator import Annotator

sns.set_theme(style="ticks", palette="deep", rc=custom_params)
p2 = pop2.mean()
p1 = pop1.mean()

df_mad_b = pd.DataFrame(TSxy2.loc[:,:,dates[0]].to_pandas().mean(axis=1),columns=["DI"])
df_mad_b['event'] = "start"
df_mad_b['place'] = "Madrid"
df_mad_a = pd.DataFrame(TSxy2.loc[:,:,dates[1]].to_pandas().mean(axis=1),columns=["DI"])
df_mad_a['event'] = "end"
df_mad_a['place'] = "Madrid"

df_bcn_b = pd.DataFrame(TSxy.loc[:,:,dates[0]].to_pandas().mean(axis=1),columns=["DI"])
df_bcn_b['event'] = "start"
df_bcn_b['place'] = "Catalunya"
df_bcn_a = pd.DataFrame(TSxy.loc[:,:,dates[1]].to_pandas().mean(axis=1),columns=["DI"])
df_bcn_a['event'] = "end"
df_bcn_a['place'] = "Catalunya"

df_policy_comp  = pd.concat([df_mad_b, df_mad_a, df_bcn_b, df_bcn_a])


fig, ax = plt.subplots(1,1, figsize=(5,3), dpi=300)

plotting_parameters = {
    "data": df_policy_comp,
    "x" : "place",
    "y": "DI",
    "hue": "event"
}

pairs = [[("Catalunya","start"),("Catalunya","end")], [("Madrid", "start"), ("Madrid", "end")]]

x = TSxy.loc[:,:,dates[0]].to_pandas().mean(axis=1)
y = TSxy.loc[:,:,dates[1]].to_pandas().mean(axis=1)

z = TSxy2.loc[:,:,dates[0]].to_pandas().mean(axis=1)
w = TSxy2.loc[:,:,dates[1]].to_pandas().mean(axis=1)

pvalues = ["{:.1e}".format(mannwhitneyu(x, y).pvalue), "{:.1e}".format(mannwhitneyu(z, w).pvalue)]
print(pvalues)
pvalues = ["****", "**"]

sns.set_palette(["white", "grey"])
sns.boxplot(**plotting_parameters, showfliers=False, ax=ax)

annotator = Annotator(ax, pairs, **plotting_parameters)
annotator.set_custom_annotations(pvalues)
annotator.annotate()

ax.legend(loc=2)
ax.set_xlabel("")
ax.set_ylabel(r"DI$^+$")

fig.savefig("../figures/supp/FigureS15.tiff", dpi=300)